# Import libraries and models

In [1]:
from notebooks.functions.verbs_dictionary import *

# Test

In [2]:
verbo = "aullar"
conjugations = scrape_verb_conjugations(verbo)
conjugations = move_inf_first_column(conjugations)
conjugations.head()

,Infinitivo_Simple,Indicativo_Presente,Indicativo_Pretérito perfecto compuesto,Indicativo_Pretérito imperfecto,Indicativo_Pretérito pluscuamperfecto,Indicativo_Pretérito perfecto simple,Indicativo_Pretérito anterior,Indicativo_Futuro,Indicativo_Futuro perfecto,Indicativo_Condicional,...,Subjuntivo_Pretérito imperfecto 2,Subjuntivo_Pretérito pluscuamperfecto 2,Subjuntivo_Futuro,Subjuntivo_Futuro perfecto,Imperativo_Imperativo,Imperativo_Imperativo negativo,Infinitivo_Compuesto,Gerundio_Simple,Gerundio_Compuesto,Participio_Pasado
0,aullar,"[aullo, aullas, aulla, aullamos, aullais, aullan]","[he aullado, has aullado, ha aullado, hemos au...","[aullaba, aullabas, aullaba, aullabamos, aulla...","[habia aullado, habias aullado, habia aullado,...","[aulle, aullaste, aullo, aullamos, aullasteis,...","[hube aullado, hubiste aullado, hubo aullado, ...","[aullare, aullaras, aullara, aullaremos, aulla...","[habre aullado, habras aullado, habra aullado,...","[aullaria, aullarias, aullaria, aullariamos, a...",...,"[aullase, aullases, aullase, aullasemos, aulla...","[hubiese aullado, hubieses aullado, hubiese au...","[aullare, aullares, aullare, aullaremos, aulla...","[hubiere aullado, hubieres aullado, hubiere au...","[aulla, aulle, aullemos, aullad, aullen]","[aulles, aulle, aullemos, aulleis, aullen]",[haber aullado],[aullando],[habiendo aullado],[aullado]


# Main

In [3]:
import random
verbs = read_verbs("./data/diccionario/verbos-espanol.txt")
# Take 20 random verbs not in order using random.sample
# verbs = random.sample(verbs, 50)
# Take the first 10 verbs
verbs = verbs[:10]
# Read the dictionary
verbs_df = read_dictionary("./data/diccionario/df_structured_verbos.csv")
# Create the dictionary
verbs_df, errores = create_dictionary(verbs, verbs_df)
# Create all conjugations list
all_conjugations = create_all_conjugations_list(verbs_df)
# Write the dictionary
write_dictionary(verbs_df, "./data/diccionario/df_structured_verbos.csv")
# Save all conjugations list
with open("./data/diccionario/all_conjugations.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(all_conjugations))
# Save errors
with open("./data/diccionario/errores.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(errores))

Processing desapadrinar:   0%|          | 0/20 [00:00<?, ?it/s]

Processing desbaratar: 100%|██████████| 20/20 [00:11<00:00,  1.69it/s] 


In [4]:
print(all_conjugations)

['acarralar', 'acarralo', 'acarralas', 'acarrala', 'acarralamos', 'acarralais', 'acarralan', 'he acarralado', 'has acarralado', 'ha acarralado', 'hemos acarralado', 'habeis acarralado', 'han acarralado', 'acarralaba', 'acarralabas', 'acarralaba', 'acarralabamos', 'acarralabais', 'acarralaban', 'habia acarralado', 'habias acarralado', 'habia acarralado', 'habiamos acarralado', 'habiais acarralado', 'habian acarralado', 'acarrale', 'acarralaste', 'acarralo', 'acarralamos', 'acarralasteis', 'acarralaron', 'hube acarralado', 'hubiste acarralado', 'hubo acarralado', 'hubimos acarralado', 'hubisteis acarralado', 'hubieron acarralado', 'acarralare', 'acarralaras', 'acarralara', 'acarralaremos', 'acarralareis', 'acarralaran', 'habre acarralado', 'habras acarralado', 'habra acarralado', 'habremos acarralado', 'habreis acarralado', 'habran acarralado', 'acarralaria', 'acarralarias', 'acarralaria', 'acarralariamos', 'acarralariais', 'acarralarian', 'habria acarralado', 'habrias acarralado', 'habr

# Time test

In [5]:
# # Verify "abacore" is a verb
# verb = "aullando"
# # Temporalize the process
# start_time = time.time()
# # print(verify_is_verb(verb, all_conjugations, diccionario))
# # print("--- %s seconds ---" % (time.time() - start_time))

# # Create a job offer text
# text = "si te critico es porque estas hurtando y criticando a los demás"
# verbos = []
# start_time = time.time()
# for word in text.split(" "):
#     verbo = verify_is_verb(word, all_conjugations, dict_verbs)
#     if verbo != "":
#         verbos.append(verbo)
# print(verbos)
# print("--- %s seconds ---" % (time.time() - start_time))